# Partição dos dados

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=070de50375c36a22678ab2f14b51a0b8ae9465a39a302b8dc4d53c5d292ecc77
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import pyspark
#conexão
from pyspark.sql import SparkSession 
from pyspark.sql.functions import max, count, col, asc, desc
#plots
import pandas as pd
import plotly.express as px

#### Transformando o arquivo em um dataframe

In [44]:
spark = SparkSession.builder\
      .master("local")\
      .appName("Particao")\
      .getOrCreate()

In [45]:
df = spark.read.csv("/content/2022-2023 Football Player Stats.csv", header= True, inferSchema = True, sep=";")

In [46]:
df.show()

+---+-----------------+------+----+-------------+--------------+---+----+---+------+----+----+-----+-----+----+----+----+-----+-------+-----+-----+-----+---------+---------+----------+----------+-------------+---------+---------+----------+---------+---------+----------+---------+---------+----------+-------+------+------+----+-----+-------+------+-------+-------+-----+----+----+------+----+----+----+-----+-----+------+------+---------+----+-----------+-----------+-------+-----+------+------+----+-----------+-----------+-------+-----+------+------+----+------+---------+---------+---------+------+---------+-------+----------+------+-----+-------+----+-------+----+----+-------+---------+---------+---------+---------+---------+-------+-----+-----+------+-----+------+-------+----------+----------+-------+------+----+------+------+----+-------+----+----+-----+----+----+----+----+----+-----+-----+----+-----+------+-------+-------+
| Rk|           Player|Nation| Pos|        Squad|          Co

In [47]:
df.filter(df['squad'] == 'Manchester City').show()

+----+---------------+------+----+---------------+--------------+---+----+---+------+----+----+-----+-----+----+-----+----+-----+-------+-----+-----+-----+---------+---------+----------+----------+-------------+---------+---------+----------+---------+---------+----------+---------+---------+----------+-------+------+------+----+-----+-------+------+-------+-------+-----+----+----+------+----+----+----+-----+-----+------+------+---------+----+-----------+-----------+-------+-----+------+------+----+-----------+-----------+-------+-----+------+------+----+------+---------+---------+---------+------+---------+-------+----------+------+-----+-------+----+-------+----+----+-------+---------+---------+---------+---------+---------+-------+-----+-----+------+-----+------+-------+----------+----------+-------+------+----+------+------+----+-------+----+----+-----+----+----+----+----+----+-----+-----+----+-----+------+-------+-------+
|  Rk|         Player|Nation| Pos|          Squad|         

In [ ]:
base = df.toPandas()
base['Pos'].value_counts()

#### categorizando as posições para realizar uma partição de dados que atendam aos nossos objetivos, separando eles por Defensor, Meio_Campo, Goleiro e Atacante

In [49]:
# o script abaixo foi criado para simplificar as posições dos jogadores
encoder = {'GK' : 'Goleiro',
#defensores
'DF':'Defensor','DFFW' : 'Defensor',
#meias
'MF': 'Meio_Campo', 'DFMF': 'Meio_Campo', 'MFFW': 'Meio_Campo', 'MFDF': 'Meio_Campo',
#atacantes
'FW': 'Atacante', 'FWDF' : 'Atacante', 'FWMF' : 'Atacante'}
Players = base.copy()
Players['funcao'] = Players.Pos.replace(encoder)

In [50]:
Players.to_csv('/content/Players_2022-2023.csv')
df2 = spark.read.csv('/content/Players_2022-2023.csv', header = True, inferSchema= True)

Logo após criar uma coluna para simplificar as posições dos jogadores é realizada a partição de dados a partir da coluna funcao.

In [51]:
df2.write.partitionBy('funcao').mode('overwrite').save('/content/jogadores-posicao')